<h2 align="center">Codebasics DL Course: Vehicle Damage Detection Project: Hyperparameter Tunning</h2>

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load Data

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2300

In [5]:
2300*0.75

1725.0

In [6]:
class_names = dataset.classes
class_names 

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

6

In [8]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1725, 575)

In [9]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model Training & Hyperparameter Tunning

In [11]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [13]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2026-01-01 06:46:49,620] A new study created in memory with name: no-name-8ee00d9a-d6ee-43fb-ada0-93970197c0ec
[I 2026-01-01 06:51:48,823] Trial 0 finished with value: 80.0 and parameters: {'lr': 0.0021646785894540426, 'dropout_rate': 0.3850446588507137}. Best is trial 0 with value: 80.0.


Execution time: 297.01030015945435 seconds


[I 2026-01-01 06:56:11,086] Trial 1 finished with value: 73.73913043478261 and parameters: {'lr': 3.843625885367595e-05, 'dropout_rate': 0.21086418458717465}. Best is trial 0 with value: 80.0.


Execution time: 261.8958296775818 seconds


[I 2026-01-01 07:00:04,309] Trial 2 finished with value: 48.0 and parameters: {'lr': 1.0542150333697215e-05, 'dropout_rate': 0.38514637116347794}. Best is trial 0 with value: 80.0.


...

Execution time: 312.6669762134552 seconds


[I 2026-01-01 07:27:11,299] Trial 9 pruned. 
[I 2026-01-01 07:28:56,026] Trial 10 pruned. 
[I 2026-01-01 07:30:41,011] Trial 11 pruned. 
[I 2026-01-01 07:35:55,607] Trial 12 finished with value: 80.52173913043478 and parameters: {'lr': 0.0001636526036761608, 'dropout_rate': 0.47239778077656036}. Best is trial 4 with value: 82.95652173913044.


Execution time: 314.051144361496 seconds


[I 2026-01-01 07:37:42,369] Trial 13 pruned. 
[I 2026-01-01 07:39:26,782] Trial 14 pruned. 
[I 2026-01-01 07:41:11,991] Trial 15 pruned. 
[I 2026-01-01 07:42:57,249] Trial 16 pruned. 
[I 2026-01-01 07:44:42,112] Trial 17 pruned. 
[I 2026-01-01 07:46:26,968] Trial 18 pruned. 
[I 2026-01-01 07:51:41,109] Trial 19 finished with value: 79.82608695652173 and parameters: {'lr': 0.0010913536639451983, 'dropout_rate': 0.2967433512711031}. Best is trial 4 with value: 82.95652173913044.


Execution time: 313.5951449871063 seconds


In [14]:
study.best_params

{'lr': 0.00847462143232574, 'dropout_rate': 0.26169167065850935}

In the first run it gave me best results with lr=0.005 and dropout=0.2. In the second run it gave different results as you see it above. Both the runs are not making much difference in terms of accuracy hence I will just use the results from the first run